# Depth Estimation Research Colab Notebook
## Автор: Кирилина Антон, Милёшин Илья, Корнаев Алексей

### Основной ноутбук для запуска экспериментов по сравнению моделей depth estimation.

In [ ]:
# Установка необходимых библиотек
!pip install torch torchvision timm opencv-python matplotlib numpy tqdm --quiet


In [ ]:
# Импорты
import torch
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import requests
from io import BytesIO
from tqdm import tqdm
import os


## Загрузка моделей

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
midas_transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')

midas.to(device)
midas.eval()


## Функция инференса depth карты

In [ ]:
def estimate_depth(image): 
    transform = midas_transforms.small_transform
    input_batch = transform(image).to(device)

    with torch.no_grad():
        prediction = midas(input_batch)
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=image.size[::-1],
            mode='bicubic',
            align_corners=False,
        ).squeeze()

    return prediction.cpu().numpy()


## Демонстрация на одной картинке

In [ ]:
url = "https://raw.githubusercontent.com/intel-isl/MiDaS/master/examples/input.jpg"
response = requests.get(url)
img = Image.open(BytesIO(response.content)).convert("RGB")

depth_map = estimate_depth(img)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title('Image')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(depth_map, cmap='inferno')
plt.title('Depth Map')
plt.axis('off')
plt.show()


## Загрузка NYU Depth v2 датасета (семпл)

In [ ]:
!wget -q https://data.vision.ee.ethz.ch/kmaninis/share/nyu_data.zip -O nyu_data.zip
!unzip -q nyu_data.zip -d nyu_data && rm nyu_data.zip


## Инференс на нескольких изображениях из датасета

In [ ]:
nyu_dir = 'nyu_data/images'
files = sorted(os.listdir(nyu_dir))[:5]

for filename in tqdm(files):
    path = os.path.join(nyu_dir, filename)
    img = Image.open(path).convert('RGB')
    depth_map = estimate_depth(img)
    
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.title('Image')
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(depth_map, cmap='inferno')
    plt.title('Depth Map')
    plt.axis('off')
    plt.show()
